In [1]:
import pandas as pd

In [2]:
df= pd.read_excel('/content/markup_df_train.xlsx')

#Загрузка разметки

In [3]:
df.head(10)

,номер видео,наименование нарушения,"сумма штрафа, руб.",время нарушения (в секундах)
0,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,18
1,akn00006_fqGg6dtL,Статья 12.16 часть 2 Поворот налево или развор...,1000,75
2,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,249
3,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,250
4,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,263
5,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,265
6,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,271
7,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,278
8,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,272
9,akn00036,Статья 12.12 часть 2 1. невыполнение требовани...,800,4


#Симуляция создания ответа от решения

Создание grounded true (файл с правильной разметкой)

In [4]:
gt = df

In [5]:
gt.to_csv('grounded_true.csv', index = False)

In [6]:
gt['наименование нарушения'].unique()

array(['Статья 12.16. часть 1 Несоблюдение требований, предписанных дорожными знаками или разметкой проезжей части дороги',
       'Статья 12.16 часть 2 Поворот налево или разворот в нарушение требований, предписанных дорожными знаками или разметкой проезжей части дороги',
       'Статья 12.17  часть 1.1 и 1.2. движение транспортных средств по полосе для маршрутных транспортных средств или остановка на указанной полосе в нарушение Правил дорожного движения ',
       'Статья 12.12 часть 2 1. невыполнение требования ПДД об остановке перед стоп-линией, обозначенной дорожными знаками или разметкой проезжей части дороги, при запрещающем сигнале светофора или запрещающем жесте регулировщика',
       'Статья 12.15 часть 4 Выезд в нарушение правил дорожного движения на полосу, предназначенную для встречного движения, при объезде препятствия, либо на трамвайные пути встречного направления, за исключением случаев, предусмотренных частью 3 настоящей статьи'],
      dtype=object)

Создание функции для рандомной генерации submission.csv

In [7]:
import numpy as np
def simul_sub(df):
  violation_list = df['наименование нарушения'].unique()
  len_base_df = len(df)

  # Процент строк для удаления и дублирования
  fraction_to_remove = 0.4
  fraction_to_duplicate = 0.3

  num_to_remove = int(len_base_df * fraction_to_remove)
  num_to_duplicate = int(len_base_df * fraction_to_duplicate)
  rows_to_remove = df.sample(num_to_remove).index
  data_dropped = df.drop(rows_to_remove)
  rows_to_duplicate = data_dropped.sample(num_to_duplicate)
  data_final = pd.concat([data_dropped, rows_to_duplicate])
  data_final = data_final.reset_index(drop=True)

  # Процент строк для изменения предсказанных секунд
  fraction_to_change = 0.5

  rows_to_change = data_final.sample(frac=fraction_to_change).index
  data_final.loc[rows_to_change, 'наименование нарушения'] = np.random.choice(violation_list, size=len(rows_to_change))
  noise_level = 10
  noise = np.random.randint(-noise_level, noise_level + 1, size=data_final.shape[0])

  data_final['время нарушения (в секундах)'] = data_final['время нарушения (в секундах)'] + noise

  return data_final

Суимуляция реального submission.csv

In [8]:
sub = simul_sub(gt.copy())

In [9]:
sub.head(10)

,номер видео,наименование нарушения,"сумма штрафа, руб.",время нарушения (в секундах)
0,akn00006_fqGg6dtL,Статья 12.16 часть 2 Поворот налево или развор...,500,27
1,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,251
2,akn00006_fqGg6dtL,Статья 12.15 часть 4 Выезд в нарушение правил ...,1500,242
3,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,254
4,akn00006_fqGg6dtL,Статья 12.16 часть 2 Поворот налево или развор...,1500,278
5,akn00036,Статья 12.12 часть 2 1. невыполнение требовани...,800,183
6,akn00077,Статья 12.12 часть 2 1. невыполнение требовани...,5000,130
7,akn00077,Статья 12.15 часть 4 Выезд в нарушение правил ...,5000,181
8,akn00078,Статья 12.17 часть 1.1 и 1.2. движение трансп...,500,239
9,akn00080,Статья 12.15 часть 4 Выезд в нарушение правил ...,1500,60


In [10]:
sub.to_csv('submission.csv', index = False)

Для каждого нарушения из gt (grounded true) мы берем ближайшее предсказание из sub (submission) для одного типа нарушения. Это одна из частей расчета метрики

In [11]:
for i, r_sub in sub.iterrows():
  #Брем строки из gt, которая имеет то же название видео и тот же класс нарушения
  video_gt = gt[(gt['номер видео'] == r_sub['номер видео']) & (gt['наименование нарушения'] == r_sub['наименование нарушения'])]
  if len(video_gt)==0: continue
  pred_sec = r_sub['время нарушения (в секундах)']
  closest_number = min(video_gt['время нарушения (в секундах)'].values, key=lambda x: abs(x - pred_sec))
  print(closest_number, pred_sec)

75 27
249 251
249 254
75 278
174 183
189 181
2 6
4 8
25 24
19 12
107 109
107 107
162 155
171 175
185 189
207 203
49 54
249 255
189 191
18 18
171 167
56 55
162 166
18 16
4 11
263 266
25 25
41 37


#Метрика

##Проверка файла на корректность

In [12]:
def correct_check(gt, sub):
  for rule in sub['наименование нарушения'].unique():
    if rule not in gt['наименование нарушения'].unique():
      raise Exception(f"Правила '{rule}' нет в разметке. Проверьте корректность его написания или убирите его. Должны остаться только те правила, которые есть в grounded true разметке")

##Функция для расчета метрики

В функции `pre_calc_score` оценивается качество предсказаний нарушений, сравнивая фактические данные (`gt`) с предсказанными (`sub`). Алгоритм работает следующим образом:

1. **Сравнение каждого нарушения**: Для каждого нарушения из `gt` подбираются аналогичные строки из `sub`, относящиеся к тому же видео и к тому же типу нарушения.

2. **Отсутствие предсказаний для нарушений**: Если предсказаний для нарушения из `gt` нет, добавляются:
   - Пропущенное предсказание (`NaN`) и метка некорректного предсказания.
   - Значение False Negative (`FN`), отражающее, что нарушения не учтены в предсказаниях.

3. **Выбор ближайшего предсказания**: Если предсказание есть, выбирается ближайшее предсказанное время нарушения к фактическому.
   - Если предсказание попадает в интервал ±5 секунд от фактического времени, то предсказание считается корректным.
   
4. **Избыточные и недостающие предсказания**: Также считаются ошибки False Positive (`FP`) и False Negative (`FN`) для каждого нарушения:
   - `FP` показывает, насколько больше предсказаний нарушений в `sub`, чем в `gt` (для выбранного видео и для выбранного типа нарушения).
   - `FN` показывает, насколько меньше предсказаний нарушений в `sub`, чем в `gt` (для выбранного видео и для выбранного типа нарушения).

5. **Подсчет финального результата (score)**:
   - Суммируется количество корректных предсказаний (`True` значений).
   - Вычитается штраф за избыточные предсказания, умноженный на 0.5.
   - Затем результат делится на общее число нарушений в `gt`, чтобы получить итоговый скор.

Формула скора выглядит так:

$
\text{score} = \frac{\text{кол-во корректных предсказаний} - (0.5 \times \text{FP})}{\text{общее количество нарушений в } gt}
$

Суть - функция метрики поощряет точные предсказания (с временным интервалом до 5 секунд от фактического) и штрафует за избыточные предсказания.

In [13]:
#Функция для подсчета некоторых важных для расчета метрики значений
def pre_calc_score(gt, sub):
  #sub - submission файл
  #gt - grounded true файл
  pred_seconds = []
  correct_predictions = []
  AE_count_rules_FP = []
  AE_count_rules_FN = []

  #Проходимся по всем нарушениям из gt
  for i, r_gt in gt.iterrows():
    #Берем строки из sub, которые относятся к одному видео и имеют нарушение того же правила, что и анализируемое нарушение в строке gt
    video_sub = sub[(sub['номер видео'] == r_gt['номер видео']) & (sub['наименование нарушения'] == r_gt['наименование нарушения'])]
    #Берем строки из gt, которые относятся к одному видео и имеют нарушение того же правила, что и анализируемое нарушение в строке gt
    video_gt = gt[(gt['номер видео'] == r_gt['номер видео']) & (gt['наименование нарушения'] == r_gt['наименование нарушения'])]

    #Обработка случая, когда есть нарушения правила в grounded true, а в submission его нарушений нет
    if len(video_sub)==0:
      #Если в предсказании не было найдено нарушеного правила, хотя на самом деле оно было, то correct_prediction = False
      pred_seconds.append(np.NaN)
      correct_predictions.append(False)

      #В предсказании было больше нарушений, чем есть на самом деле
      FP = max(0 - len(video_gt), 0) # всегда будет 0
      #В предсказании было меньше нарушений, чем есть на самом деле
      FN = abs(min(0 - len(video_gt), 0))

      AE_count_rules_FP.append(FP)
      AE_count_rules_FN.append(FN)
      continue

    true_sec = r_gt['время нарушения (в секундах)']

    #Берем ближайшую pred сенунду к true секунде для взятого правила
    pred_sec = min(video_sub['время нарушения (в секундах)'].values, key=lambda x: abs(x - true_sec))
    pred_seconds.append(pred_sec)

    #При попадание в интервал 5 секунд от фактического нарушения зачисляется предсказание
    correct_prediction = np.abs(pred_sec-true_sec)<5
    correct_predictions.append(correct_prediction)


    #В предсказании было больше нарушений, чем есть на самом деле
    FP = max(len(video_sub) - len(video_gt), 0)
    #В предсказании было меньше нарушений, чем есть на самом деле
    FN = abs(min(len(video_sub) - len(video_gt), 0))


    AE_count_rules_FP.append(FP)
    AE_count_rules_FN.append(FN)


  gt['pred_seconds'] = pred_seconds
  gt['Корректность прдсказания'] = correct_predictions
  gt['В предсказании было больше нарушений на кол-во'] = AE_count_rules_FP
  gt['В предсказании было меньше нарушений на кол-во'] = AE_count_rules_FN
  return gt

In [14]:
correct_check(gt, sub)
result_table = pre_calc_score(gt.copy(), sub.copy())

In [19]:
result_table.head(10)

,номер видео,наименование нарушения,"сумма штрафа, руб.",время нарушения (в секундах),pred_seconds,Корректность прдсказания,В предсказании было больше нарушений на кол-во,В предсказании было меньше нарушений на кол-во
0,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,18,16.0,True,2,0
1,akn00006_fqGg6dtL,Статья 12.16 часть 2 Поворот налево или развор...,1000,75,27.0,False,1,0
2,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,249,251.0,True,2,0
3,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,250,NaN,False,0,5
4,akn00006_fqGg6dtL,"Статья 12.16. часть 1 Несоблюдение требований,...",500,263,266.0,True,2,0
5,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,265,NaN,False,0,5
6,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,271,NaN,False,0,5
7,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,278,NaN,False,0,5
8,akn00006_fqGg6dtL,Статья 12.17 часть 1.1 и 1.2. движение трансп...,1500,272,NaN,False,0,5
9,akn00036,Статья 12.12 часть 2 1. невыполнение требовани...,800,4,183.0,False,0,1


Формула :

Скор для одной строки = (попадает ли предсказанное время нарушения в +-5 сек от действительного времени нарушения?) - ((Сколько избыточных попыток (на каждое нарушение дается одна попытка, остальные считаются избыточными) было сделано для предсказания нарушения?)*0.5)


Скор для все строк = (скор для строки 1 + ... + скор для строки n ) / n. Где n - длина grounded true

In [16]:
score = max(0,(result_table['Корректность прдсказания'].sum()-(result_table['В предсказании было больше нарушений на кол-во'].sum()*0.5)))/len(result_table)

In [17]:
score

0.3